In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [75]:
using StatsBase, Distributions, Plots, LinearAlgebra, Plots

In [3]:
using Revise

includet("../src/ESGModels/msvar.jl")
includet("../src/ESGModels/msvar_simulate.jl") 

In [4]:
Β1 = [.09 .3 .2; .03 .1 .6]
Σ1 = cor2cov([1 .3; .3 1], [.08, .03])
Β2 = [-.02 .35 .25; .035 .15 .63]
Σ2 = cor2cov([1 .35; .35 1], [.18, .035])


transition_matrix = [.85 .15;.2 .8]

states_zero = [.9, .1]
X = [.12, .06]


n_samples = 1000
test_regimes = simulate_regimes(states_zero, transition_matrix, n_samples, 1)
test_returns = simulate_msvar_returns(test_regimes, [Β1, Β2], [Σ1, Σ2], X, n_samples, 1)[:,:,1]'

1000×2 adjoint(::Matrix{Float64}) with eltype Float64:
  0.0594498  0.0876271
  0.0585175  0.0741366
 -0.384565   0.124349
 -0.0362412  0.084648
  0.0883398  0.0856895
  0.169986   0.100858
  0.0793413  0.0859396
  0.114779   0.0563473
  0.189468   0.0865504
  0.268172   0.109997
  ⋮          
  0.0718065  0.0827917
  0.168601   0.103735
  0.131264   0.0788732
  0.139286   0.124723
  0.205699   0.20088
  0.278883   0.197581
  0.244214   0.236575
  0.21403    0.192462
  0.229819   0.225787

In [150]:
Y = collect(test_returns[2:end,:])
T = n_samples
p = 1
X = hcat(ones(T-1), test_returns[p:end-1,:])
k= 2 #n_regimes
n_variables = 2

#Β = [randn(n_variables, n_variables+1) for _ in 1:k]

Β1 = [.06 .35 .25; .035 .2 .65]

Β2 = [-.05 .45 .15; .035 .15 .63]


#= Σ1 = cor2cov([1 .1; .1 1], [.05, .02])
Σ2 = cor2cov([1 .5; .5 1], [.10, .05]) =#

Σ = rand(InverseWishart(4, [.001 0; 0 .015]),k)

#init_regimes = rand(k) 
#init_regimes = init_regimes ./ sum(init_regimes)

t_m = [.5 .5;.5 .5]

result = expectation_maximisation(Y, X, k, Β, Σ, t_m, 100)



MSVARResult([2.375880182239418e-133 1.0; 0.0 1.0; … ; 2.7685275835337613e-192 1.0; 5.292372499730082e-163 1.0], [2.3758801822394213e-133 1.0000000000000013; 0.0 1.0000000000000013; … ; 2.7685275835337613e-192 1.0; 5.292372499730082e-163 1.0], [NaN 0.5000000000000003; NaN 0.5], Any[[0.26952981302575113 0.05431884325948068; 1.0471581565460704 0.16922505122797393; -0.058404573358992816 0.45532473221732145], [0.04328549619579321 0.028681271557113287; 0.3853279853860466 0.11947808110056227; 0.30854465512904816 0.5937076844352643]], Any[[0.0051624051178112045 0.0010575160920481602; 0.00105751609204816 0.001014943331415573], [0.011166842498007677 0.0006476057264638837; 0.0006476057264638837 0.001011745312505539]])

In [157]:
round.(result.regimes, digits=2)

999×2 Matrix{Float64}:
 0.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 ⋮    
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0

In [165]:
result.Β


2-element Vector{Any}:
 [0.26952981302575113 0.05431884325948068; 1.0471581565460704 0.16922505122797393; -0.058404573358992816 0.45532473221732145]
 [0.04328549619579321 0.028681271557113287; 0.3853279853860466 0.11947808110056227; 0.30854465512904816 0.5937076844352643]

In [168]:
r = [1. 0.; 1. 0.; 1. 0.; 0. 1.; 1. 0.]

jr = joined_regimes_probs(r, r, [.5, .5], t_m)


5×4 Matrix{Float64}:
   0.25    0.25  NaN    NaN
   0.5     0.0   NaN    NaN
   0.5     0.0   NaN    NaN
 NaN     NaN       0.5    0.0
   0.0     0.5   NaN    NaN

In [169]:
est_transition_matrix(jr, r, [.5, .5])

2×2 Matrix{Float64}:
 NaN  NaN
 NaN  NaN

2×2 Matrix{Float64}:
 0.0016  0.0
 0.0     0.0064